In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import linregress 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf   

from sklearn.metrics import mean_squared_error 

Load the dataset using the pandas package.

Write your own Python implementation of the cross-validation procedure (please do not use existing implementation). Feel free to shuffle the data so that the splits are different each time it is run.

Perform cross-validation on the pasture dataset using multiple linear regression (feel free to use sci-kit learn). In other words, train a multiple linear regression learner on the training part and evaluate it using the test part (compute RMSE, feel free to write your own implementation or use sci-kit learn). Repeat this step as the cross-validation iterations progress (make sure the number of splits is specified by the user).

Report the mean and standard deviation of the RMSE values.

In [35]:
fullpath = r"G:\My Drive\_Msc Data Science\statistics\pasture-data.csv"
data = pd.read_csv(fullpath, index_col='I')

In [111]:
# say, a 5-way split -> 
def kfold(splits, length):
    idxs = range(length)
    # np.random.shuffle(idxs)
    
    idx_bins = np.array_split(idxs, splits)
    train_test_tuples = []

    for split in range(splits):
        test_idx = idx_bins[split]
        set_of_rest_of_bins = set([bin for b in idx_bins for bin in b]) - set(test_idx)
        train_idx = np.array(list(set_of_rest_of_bins))
        train_test_tuples.append((test_idx, train_idx))
        
    assert len(train_test_tuples) == splits
    return train_test_tuples 

length = data.shape[0]

train_test_tuples = kfold(5, length)
train_test_tuples

[(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),
  array([14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
         31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
         48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
         65, 66])),
 (array([14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]),
  array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 28, 29, 30,
         31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47,
         48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
         65, 66])),
 (array([28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]),
  array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
         17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 41, 42, 43, 44, 45, 46,
         47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
         64, 65, 66])),
 (array([41, 42, 43, 44, 45, 46, 47, 48, 49, 5

In [129]:
lm = LinearRegression()
scores = []
for test_idx, train_idx in kfold(5, length):
    
    # get the split sets
    test = data.iloc[test_idx]
    train = data.iloc[train_idx]
    
    # for the test set, get X and y
    test_y = test.Y
    test_X = test.drop('Y', axis=1)
    
    # idem for the train set
    train_y = train.Y
    train_X = train.drop('Y', axis=1)
    
    # train on the train set
    lm.fit(train_X, train_y)
    
    # predict on test set
    yhat = lm.predict(test_X)
    
    # compute the fold's score
    RMSE = np.sqrt(mean_squared_error(test_y, yhat))
    scores.append(RMSE)    

f"the mean of the RMSE scores is {np.mean(scores):.2f} and the standard deviation {np.std(scores):.2f}"

'the mean of the RMSE scores is 8.59 and the standard deviation 2.69'

In [139]:
# same with sklearn
from sklearn.model_selection import cross_val_score
lm = LinearRegression()

train_data = data.drop('Y', axis=1)
train_target = data.Y
test_data = data.drop('Y', axis=1)
test_target = data.Y

cv_score = cross_val_score(lm, train_data, train_target, cv=5, scoring='neg_root_mean_squared_error') 
# default scoring is 'explained_variance'
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

f"the mean of the RMSE scores is {np.mean(cv_score):.2f} and the standard deviation {np.std(cv_score):.2f}"

'the mean of the RMSE scores is -8.59 and the standard deviation 2.69'

In [169]:
sm_lm = sm.OLS(train_target, train_data) # .assign(const=1)
results = sm_lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      Y   R-squared (uncentered):                   0.967
Model:                            OLS   Adj. R-squared (uncentered):              0.966
Method:                 Least Squares   F-statistic:                              631.7
Date:                Fri, 07 Jun 2024   Prob (F-statistic):                    1.80e-47
Time:                        15:33:34   Log-Likelihood:                         -240.00
No. Observations:                  67   AIC:                                      486.0
Df Residuals:                      64   BIC:                                      492.6
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
X1             0.8349      0.033     25.066      0.000       0.768       0.901
X2             0.4648      0.091      5.094      0.000       0.283       0.647
X3           -19.5200      8.509     -2.294      0.025     -36.519      -2.521
==============================================================================
Omnibus:                        7.916   Durbin-Watson:                   2.288
Prob(Omnibus):                  0.019   Jarque-Bera (JB):                7.601
Skew:                           0.635   Prob(JB):                       0.0224
Kurtosis:                       4.053   Cond. No.                         411.
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Note the P>|t|. 

The p-value for two-tailed test is:
$$
p = 2 \times P(T > |t|)
$$
where $ T $ follows the $ t $-distribution with $ df $ degrees of freedom.

In words: 
> The p-value is the probability of observing a 𝑡-statistic as extreme as, or more extreme than, the calculated value under the null hypothesis.

In [175]:
results.pvalues

X1    8.429609e-35
X2    3.326344e-06
X3    2.508690e-02
dtype: float64

In [178]:
2.5e-02

0.025

## same in R-style formula API

see https://faculty.washington.edu/otoomet/machinelearning-py/linear-regression.html

In [164]:
formula_lm = smf.ols(formula='Y ~ X1 + X2 + X3', data=data)
results = formula_lm.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.855
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     124.0
Date:                Fri, 07 Jun 2024   Prob (F-statistic):           2.19e-26
Time:                        15:26:10   Log-Likelihood:                -239.39
No. Observations:                  67   AIC:                             486.8
Df Residuals:                      63   BIC:                             495.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.0758      3.767     -1.082      0.283     -11.603       3.452
X1             0.8955      0.065     13.749      0.000       0.765       1.026
X2             0.4519      0.092      4.917      0.000       0.268       0.636
X3           -11.5671     11.236     -1.029      0.307     -34.020      10.886
==============================================================================
Omnibus:                        5.237   Durbin-Watson:                   2.296
Prob(Omnibus):                  0.073   Jarque-Bera (JB):                4.584
Skew:                           0.454   Prob(JB):                        0.101
Kurtosis:                       3.903   Cond. No.                         558.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""